In [1]:
# Import Libraries

import ast

import torch
import seaborn
import pandas as pd
from sklearn import metrics
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange
from tqdm.notebook import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data

In [2]:
# Import data
data = pd.read_csv('/kaggle/input/small-data-thesis/small_data_lemmatized.csv')
data

,label,text,gensim_remove,nltk_remove,gensim_token_keras,nltk_token_keras,gensim_token_nltk,nltk_token_nltk,nltk_gensim_token_keras,nltk_nltk_token_keras,nltk_gensim_token_nltk,nltk_nltk_token_nltk,nltk_stemmer_gensim_token_keras,nltk_stemmer_nltk_token_keras,nltk_stemmer_gensim_token_nltk,nltk_stemmer_nltk_token_nltk
0,Negative,not My kind of writing: I do not care for thi...,My kind writing I care author reading couple b...,kind writing care author reading couple books...,"['my', 'kind', 'writing', 'i', 'care', 'author...","['kind', 'writing', 'care', 'author', 'reading...","['My', 'kind', 'writing', 'I', 'care', 'author...","['kind', 'writing', 'care', 'author', 'reading...","['my', 'kind', 'write', 'i', 'care', 'author',...","['kind', 'write', 'care', 'author', 'reading',...","['My', 'kind', 'write', 'I', 'care', 'author',...","['kind', 'write', 'care', 'author', 'reading',...","['my', 'kind', 'write', 'i', 'care', 'author',...","['kind', 'write', 'care', 'author', 'read', 'c...","['my', 'kind', 'write', 'i', 'care', 'author',...","['kind', 'write', 'care', 'author', 'read', 'c..."
1,Negative,Just in case you didn't know what you were wa...,Just case didnt know watching The Beverly Hill...,case nt know watching Beverly Hillbillies C...,"['just', 'case', 'didnt', 'know', 'watching', ...","['case', 'nt', 'know', 'watching', 'beverly', ...","['Just', 'case', 'didnt', 'know', 'watching', ...","['case', 'nt', 'know', 'watching', 'Beverly', ...","['just', 'case', 'didnt', 'know', 'watch', 'th...","['case', 'nt', 'know', 'watch', 'beverly', 'hi...","['Just', 'case', 'didnt', 'know', 'watch', 'Th...","['case', 'nt', 'know', 'watch', 'Beverly', 'Hi...","['just', 'case', 'didnt', 'know', 'watch', 'th...","['case', 'nt', 'know', 'watch', 'beverli', 'hi...","['just', 'case', 'didnt', 'know', 'watch', 'th...","['case', 'nt', 'know', 'watch', 'beverli', 'hi..."
2,Negative,An unimpressive let down: I read Lord of the ...,An unimpressive let down I read Lord Flies I h...,unimpressive let read Lord Flies honestly say...,"['an', 'unimpressive', 'let', 'down', 'i', 're...","['unimpressive', 'let', 'read', 'lord', 'flies...","['An', 'unimpressive', 'let', 'down', 'I', 're...","['unimpressive', 'let', 'read', 'Lord', 'Flies...","['an', 'unimpressive', 'let', 'down', 'i', 're...","['unimpressive', 'let', 'read', 'lord', 'fly',...","['An', 'unimpressive', 'let', 'down', 'I', 're...","['unimpressive', 'let', 'read', 'Lord', 'Flies...","['an', 'unimpress', 'let', 'down', 'i', 'read'...","['unimpress', 'let', 'read', 'lord', 'fli', 'h...","['an', 'unimpress', 'let', 'down', 'i', 'read'...","['unimpress', 'let', 'read', 'lord', 'fli', 'h..."
3,Positive,A fine addition to the Clapton catalogue: Eri...,A fine addition Clapton catalogue Eric Clapton...,fine addition Clapton catalogue Eric Clapton ...,"['a', 'fine', 'addition', 'clapton', 'catalogu...","['fine', 'addition', 'clapton', 'catalogue', '...","['A', 'fine', 'addition', 'Clapton', 'catalogu...","['fine', 'addition', 'Clapton', 'catalogue', '...","['a', 'fine', 'addition', 'clapton', 'catalogu...","['fine', 'addition', 'clapton', 'catalogue', '...","['A', 'fine', 'addition', 'Clapton', 'catalogu...","['fine', 'addition', 'Clapton', 'catalogue', '...","['a', 'fine', 'addit', 'clapton', 'catalogu', ...","['fine', 'addit', 'clapton', 'catalogu', 'eric...","['a', 'fine', 'addit', 'clapton', 'catalogu', ...","['fine', 'addit', 'clapton', 'catalogu', 'eric..."
4,Negative,"Horribly-written, self-important, whiny mean-...",Horriblywritten selfimportant whiny meanspirit...,Horriblywritten selfimportant whiny meanspir...,"['horriblywritten', 'selfimportant', 'whiny', ...","['horriblywritten', 'selfimportant', 'whiny', ...","['Horriblywritten', 'selfimportant', 'whiny', ...","['Horriblywritten', 'selfimportant', 'whiny', ...","['horriblywritten', 'selfimportant', 'whiny', ...","['horriblywritten', 'selfimportant', 'whiny', ...","['Horriblywritten', 'selfimportant', 'whiny', ...","['Horriblywritten', 'self

In [3]:
# Find and replace values
data["label"] = data["label"].str.replace("Negative", "0")
data["label"] = data["label"].str.replace("Positive", "1")

In [4]:
# Keep only 50000 reviews: 25000 positive and 25000 negative
small_data_positive = data[data['label']=='1'].head(25000)
small_data_negative = data[data['label']=='0'].head(25000)

# Create a new dataframe
small_data = pd.concat([small_data_positive, small_data_negative])
small_data = small_data.sample(frac=1, ignore_index=True)

In [5]:
# Display sample data
small_data

,label,text,gensim_remove,nltk_remove,gensim_token_keras,nltk_token_keras,gensim_token_nltk,nltk_token_nltk,nltk_gensim_token_keras,nltk_nltk_token_keras,nltk_gensim_token_nltk,nltk_nltk_token_nltk,nltk_stemmer_gensim_token_keras,nltk_stemmer_nltk_token_keras,nltk_stemmer_gensim_token_nltk,nltk_stemmer_nltk_token_nltk
0,0,Awesome shoe... poor service: I was pretty di...,Awesome shoe poor service I pretty disappointe...,Awesome shoe poor service pretty disappointe...,"['awesome', 'shoe', 'poor', 'service', 'i', 'p...","['awesome', 'shoe', 'poor', 'service', 'pretty...","['Awesome', 'shoe', 'poor', 'service', 'I', 'p...","['Awesome', 'shoe', 'poor', 'service', 'pretty...","['awesome', 'shoe', 'poor', 'service', 'i', 'p...","['awesome', 'shoe', 'poor', 'service', 'pretty...","['Awesome', 'shoe', 'poor', 'service', 'I', 'p...","['Awesome', 'shoe', 'poor', 'service', 'pretty...","['awesom', 'shoe', 'poor', 'servic', 'i', 'pre...","['awesom', 'shoe', 'poor', 'servic', 'pretti',...","['awesom', 'shoe', 'poor', 'servic', 'i', 'pre...","['awesom', 'shoe', 'poor', 'servic', 'pretti',..."
1,1,Good first exposure to Nigerian highlife musi...,Good exposure Nigerian highlife music After li...,Good first exposure Nigerian highlife music ...,"['good', 'exposure', 'nigerian', 'highlife', '...","['good', 'first', 'exposure', 'nigerian', 'hig...","['Good', 'exposure', 'Nigerian', 'highlife', '...","['Good', 'first', 'exposure', 'Nigerian', 'hig...","['good', 'exposure', 'nigerian', 'highlife', '...","['good', 'first', 'exposure', 'nigerian', 'hig...","['Good', 'exposure', 'Nigerian', 'highlife', '...","['Good', 'first', 'exposure', 'Nigerian', 'hig...","['good', 'exposur', 'nigerian', 'highlif', 'mu...","['good', 'first', 'exposur', 'nigerian', 'high...","['good', 'exposur', 'nigerian', 'highlif', 'mu...","['good', 'first', 'exposur', 'nigerian', 'high..."
2,1,A Very Helpful Book!: I bought this book beca...,A Very Helpful Book I bought book son beginnin...,Helpful Book bought book son beginning 3rd g...,"['a', 'very', 'helpful', 'book', 'i', 'bought'...","['helpful', 'book', 'bought', 'book', 'son', '...","['A', 'Very', 'Helpful', 'Book', 'I', 'bought'...","['Helpful', 'Book', 'bought', 'book', 'son', '...","['a', 'very', 'helpful', 'book', 'i', 'bought'...","['helpful', 'book', 'bought', 'book', 'son', '...","['A', 'Very', 'Helpful', 'Book', 'I', 'bought'...","['Helpful', 'Book', 'bought', 'book', 'son', '...","['a', 'veri', 'help', 'book', 'i', 'bought', '...","['help', 'book', 'bought', 'book', 'son', 'beg...","['a', 'veri', 'help', 'book', 'i', 'bought', '...","['help', 'book', 'bought', 'book', 'son', 'beg..."
3,1,perfect: Philips DVP5140 Multiformat DVD Play...,perfect Philips DVP5140 Multiformat DVD Player...,perfect Philips DVP5140 Multiformat DVD Playe...,"['perfect', 'philips', 'dvp5140', 'multiformat...","['perfect', 'philips', 'dvp5140', 'multiformat...","['perfect', 'Philips', 'DVP5140', 'Multiformat...","['perfect', 'Philips', 'DVP5140', 'Multiformat...","['perfect', 'philip', 'dvp5140', 'multiformat'...","['perfect', 'philip', 'dvp5140', 'multiformat'...","['perfect', 'Philips', 'DVP5140', 'Multiformat...","['perfect', 'Philips', 'DVP5140', 'Multiformat...","['perfect', 'philip', 'dvp5140', 'multiformat'...","['perfect', 'philip', 'dvp5140', 'multiformat'...","['perfect', 'philip', 'dvp5140', 'multiformat'...","['perfect', 'philip', 'dvp5140', 'multiformat'..."
4,1,Life lessons: It is the first book that I rea...,Life lessons It book I read impression me I 13...,Life lessons first book read really made impr...,"['life', 'lessons', 'it', 'book', 'i', 'read',...","['life', 'lessons', 'first', 'book', 'read', '...","['Life', 'lessons', 'It', 'book', 'I', 'read',...","['Life', 'lessons', 'first', 'book', 'read', '...","['life', 'lesson', 'it', 'book', 'i', 'read', ...","['life', 'lesson', 'first', 'book', 'read', 'r...","['Life', 'lesson', 'It', 'book', 'I', 'read', ...","['Life', 'lesson', 'first', 'book', 'read', 'r...","['life', '

In [6]:
# Loading datasets

reviews = small_data['text'].values.tolist()
sentiments = small_data['label'].astype(int).values.tolist()

# Functions

In [7]:
# Create a function that encodes data based on the model tokenizer and returns data loaders
# INPUT: list of reviews, list of sentiments, tokenizer
# OUTPUT: training and validation dataloaders

def encode_data_deep_models(reviews, sentiments, TOKENIZER):
    
    # Encoding batch
    encoded_batch = TOKENIZER.batch_encode_plus(reviews,
                                                add_special_tokens=True,
                                                padding=True,
                                                truncation=True,
                                                return_attention_mask = True,
                                                return_tensors = 'pt')
    
    # Transform list to tensor
    sentiments = torch.tensor(sentiments)

    # Use only 80% of data fro training and 20% for testing
    split_border = int(len(sentiments)*0.8)

    # Create tensor datasets for training and validation
    train_dataset = TensorDataset(
        encoded_batch['input_ids'][:split_border],
        encoded_batch['attention_mask'][:split_border],
        sentiments[:split_border])
    validation_dataset = TensorDataset(
        encoded_batch['input_ids'][split_border:],
        encoded_batch['attention_mask'][split_border:],
        sentiments[split_border:])

    # Define batch size
    batch_size = 16

    # Create DataLoader from tensor datasets
    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset),
                batch_size = batch_size)

    valid_dataloader = DataLoader(
                validation_dataset,
                sampler = SequentialSampler(validation_dataset),
                batch_size = batch_size)
    
    return train_dataloader, valid_dataloader

In [8]:
# Create a function that returns the mean of a list
# INPUT: list
# OUTPUT: mean of that list

def mean(lst):
    return sum(lst) / len(lst)

In [9]:
# Create a function for the training of the model
# INPUT: model, optimizer, number of epochs, training dataloader and validation dataloader
# OUTPUT: None

def train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader):
    
    # Train on GPU using CUDA
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Initialise lists for storing losses and accuracies
    train_accuracies = []
    train_losses = []
    validation_accuracies = []
    validation_losses = []
    test_accuracies = []
    learning_rates = []
    
    # Accuracy threshold
    max_validation_accuracy = 0
    
    # Import the model on GPU memory
    torch.cuda.memory_summary(device=None, abbreviated=False)
    model = model.to(device)

    # Training loop
    for epoch in range(0, epochs):

        print("")
        print(f'########## Epoch {epoch+1} / {epochs} ##########')
        print('Training...')

        # Model in training mode
        model.train()

        train_batch_losses = []
        correct = 0
        total = 0

        # For each batch of reviews
        for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):

            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch

            # Put the gradient to 0
            optimizer.zero_grad()

            # Give data to the model and retrieve the output
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            
            # Recover predictions and determine the accuracy
            _, predicted = torch.max(outputs[1], 1)
            total += b_labels.size(0)
            correct += (predicted == b_labels).sum().item()

            loss = outputs[0]
            logits = outputs[1]
            
            # Save the loss
            train_batch_losses.append(loss.item())

            # Backpropagtion
            loss.backward()

            # Update optimizer
            optimizer.step()
            learning_rates.append(optimizer.param_groups[0]["lr"])
        
        # Calculate training accuracy
        train_accuracy = 100 * correct / total

        train_losses.append(mean(train_batch_losses))
        train_accuracies.append(train_accuracy)

        # Same for validation set
        validation_batch_losses = []
        correct = 0
        total = 0
        
        # Model in evaluation mode
        model.eval()
        
        for step, batch in tqdm(enumerate(valid_dataloader), total=len(valid_dataloader)):
            with torch.no_grad():   # no_grad is just for reducing memory consumption when you are sure that you will not call Tensor.backward(), that you are not going to use the gradient.
                batch = tuple(t.to(device) for t in batch)
                # Unpack the inputs from our dataloader
                b_input_ids, b_input_mask, b_labels = batch
                outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

                _, predicted = torch.max(outputs[1], 1)
                total += b_labels.size(0)
                correct += (predicted == b_labels).sum().item()

            validation_loss = outputs[0]
            validation_batch_losses.append(validation_loss.item())
        
        # Calculate validation accuracy
        validation_accuracy = 100 * correct / total
        
        # Store the model with the highest validation accuracy
        if validation_accuracy > max_validation_accuracy:
            max_validation_accuracy = validation_accuracy
            torch.save(model.state_dict(), "bestmodel.pt")

        validation_losses.append(mean(validation_batch_losses))
        validation_accuracies.append(validation_accuracy)
        
        # Display results for the epoch
        print(f"Epoch : {epoch}  Train accuracy : {train_accuracy}   Train Loss : {mean(train_batch_losses)}    Validation accuracy : {validation_accuracy}   Validation Loss : {mean(validation_batch_losses)}")

# BERT

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification

# Loading the tokenizer from Bert
TOKENIZER = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [11]:
# Determine training and validation dataloaders
train_dataloader, valid_dataloader = encode_data_deep_models(reviews, sentiments, TOKENIZER)

In [12]:
# Train model for 2 epochs with Adam optimizer
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i


########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 89.8025   Train Loss : 0.26069773636125027    Validation accuracy : 90.44   Validation Loss : 0.24584586267471314

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 82.08   Train Loss : 0.3377632880035788    Validation accuracy : 49.49   Validation Loss : 0.7013474775314331


In [13]:
# Train model for 2 epochs with AdamW optimizer
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2)

optimizer = torch.optim.AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i


########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 93.485   Train Loss : 0.17125254941601306    Validation accuracy : 95.42   Validation Loss : 0.13815365067645907

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 97.1625   Train Loss : 0.083984284385317    Validation accuracy : 95.48   Validation Loss : 0.13547999141365288


# BERT-SA

In [14]:
from transformers import BertTokenizer, BertForSequenceClassification

# Loading the tokenizer from Bert-SA
TOKENIZER = BertTokenizer.from_pretrained(
    'textattack/bert-base-uncased-yelp-polarity',
    do_lower_case=True)

In [15]:
# Determine training and validation dataloaders
train_dataloader, valid_dataloader = encode_data_deep_models(reviews, sentiments, TOKENIZER)

In [16]:
# Train model for 2 epochs with Adam optimizer
model = BertForSequenceClassification.from_pretrained(
    'textattack/bert-base-uncased-yelp-polarity',
    num_labels = 2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)


########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 92.665   Train Loss : 0.19152492353674025    Validation accuracy : 92.84   Validation Loss : 0.1983584648348391

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 94.685   Train Loss : 0.14615887590441853    Validation accuracy : 92.85   Validation Loss : 0.21045805053636432


In [17]:
# Train model for 2 epochs with AdamW optimizer
model = BertForSequenceClassification.from_pretrained(
    'textattack/bert-base-uncased-yelp-polarity',
    num_labels = 2)

optimizer = torch.optim.AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)


########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 94.3475   Train Loss : 0.1512461171083851    Validation accuracy : 94.93   Validation Loss : 0.13633157751783728

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 97.6225   Train Loss : 0.07199757177294232    Validation accuracy : 94.96   Validation Loss : 0.160656768588908


# DistilBERT

In [18]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Loading the tokenizer from DistilBERT
TOKENIZER = DistilBertTokenizer.from_pretrained(
    'distilbert-base-uncased',
    do_lower_case=True)

In [19]:
# Determine training and validation dataloaders
train_dataloader, valid_dataloader = encode_data_deep_models(reviews, sentiments, TOKENIZER)

In [20]:
# Train model for 2 epochs with Adam optimizer
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels = 2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.


########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 91.1425   Train Loss : 0.22555312088355423    Validation accuracy : 93.18   Validation Loss : 0.19158042245805262

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 94.8125   Train Loss : 0.14353002109611407    Validation accuracy : 92.12   Validation Loss : 0.2030816372551024


In [21]:
# Train model for 2 epochs with AdamW optimizer
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels = 2)

optimizer = torch.optim.AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.


########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 92.86   Train Loss : 0.18384377601109445    Validation accuracy : 94.73   Validation Loss : 0.14397168650999664

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 96.5225   Train Loss : 0.09939817979941144    Validation accuracy : 94.35   Validation Loss : 0.16661776690315455


# AlBERT

In [22]:
from transformers import AlbertForSequenceClassification, AlbertTokenizer

# Loading the tokenizer from AlBERT
TOKENIZER = AlbertTokenizer.from_pretrained(
    'textattack/albert-base-v2-imdb',
    do_lower_case=True)

In [23]:
# Determine training and validation dataloaders
train_dataloader, valid_dataloader = encode_data_deep_models(reviews, sentiments, TOKENIZER)

In [24]:
# Train model for 2 epochs with Adam optimizer
model = AlbertForSequenceClassification.from_pretrained(
    'textattack/albert-base-v2-imdb',
    num_labels = 2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)


########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 69.5625   Train Loss : 0.5056160506721586    Validation accuracy : 49.49   Validation Loss : 0.6940037293434143

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 49.62   Train Loss : 0.7055128307580948    Validation accuracy : 50.51   Validation Loss : 0.6940771789550781


In [25]:
# Train model for 2 epochs with AdamW optimizer
model = AlbertForSequenceClassification.from_pretrained(
    'textattack/albert-base-v2-imdb',
    num_labels = 2)

optimizer = torch.optim.AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)


########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 94.43   Train Loss : 0.14986347327884286    Validation accuracy : 95.16   Validation Loss : 0.13413529737070204

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 96.0775   Train Loss : 0.10703382148058153    Validation accuracy : 94.78   Validation Loss : 0.1497174129344523


# RoBERTa

In [26]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

# Loading the tokenizer from RoBERTa
TOKENIZER = RobertaTokenizer.from_pretrained(
    'roberta-base',
    do_lower_case=True)

In [27]:
# Determine training and validation dataloaders
train_dataloader, valid_dataloader = encode_data_deep_models(reviews, sentiments, TOKENIZER)

In [28]:
# Train model for 2 epochs with Adam optimizer
model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels = 2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should pr


########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 50.02   Train Loss : 0.6969097863197327    Validation accuracy : 49.49   Validation Loss : 0.6967290449142456

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 49.9275   Train Loss : 0.6956244893550872    Validation accuracy : 49.49   Validation Loss : 0.6934914374351502


In [29]:
# Train model for 2 epochs with AdamW optimizer
model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels = 2)

optimizer = torch.optim.AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should pr


########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 94.105   Train Loss : 0.1591321494435426    Validation accuracy : 95.9   Validation Loss : 0.12063948825746774

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 96.6925   Train Loss : 0.0954173900400754    Validation accuracy : 95.47   Validation Loss : 0.15093905536234378


# RoBERTa-SA

In [30]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

# Loading the tokenizer from RoBERTa-SA
TOKENIZER = RobertaTokenizer.from_pretrained(
    'cardiffnlp/twitter-roberta-base-emotion',
    do_lower_case=True)

In [31]:
# Determine training and validation dataloaders
train_dataloader, valid_dataloader = encode_data_deep_models(reviews, sentiments, TOKENIZER)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [32]:
# Train model for 2 epochs with Adam optimizer
model = RobertaForSequenceClassification.from_pretrained(
    'cardiffnlp/twitter-roberta-base-emotion',
    num_labels = 2, ignore_mismatched_sizes=True)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 50.23   Train Loss : 0.6989615890026093    Validation accuracy : 49.49   Validation Loss : 0.6973719287872314

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 49.695   Train Loss : 0.695327663564682    Validation accuracy : 50.51   Validation Loss : 0.6931024823188782


In [33]:
# Train model for 2 epochs with AdamW optimizer
model = RobertaForSequenceClassification.from_pretrained(
    'cardiffnlp/twitter-roberta-base-emotion',
    num_labels = 2, ignore_mismatched_sizes=True)

optimizer = torch.optim.AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)
epochs = 2

train_deep_models_optimizer(model, optimizer, epochs, train_dataloader, valid_dataloader)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([4, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([4]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



########## Epoch 1 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 0  Train accuracy : 94.5375   Train Loss : 0.15259479473968968    Validation accuracy : 95.77   Validation Loss : 0.12999838447719814

########## Epoch 2 / 2 ##########
Training...


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : 1  Train accuracy : 96.83   Train Loss : 0.09031856554364785    Validation accuracy : 95.93   Validation Loss : 0.12576257358118892
